In [ ]:
from keras.datasets import cifar10
from matplotlib import pyplot

(x_train, x_y), (x_test, y_test) = cifar10.load_data()

pyplot.imshow(x_train[0])

In [ ]:
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# discriminator architecture
def discriminator_model(in_shape=(32,32,3)):
    model = Sequential()
    
    model.add(Conv2D(64, 3, padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, 3, strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, 3, strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(256, 3, strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0002), metrics=["accuracy"])
    return model

# generator architecture
def generator_model(latent_dim):
    model = Sequential()
    
    n_nodes = 256 *4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))

    model.add(Conv2DTranspose(128, 4, strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(128, 4, strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, 4, strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (3,3), activation="tanh", padding="same"))
    return model

# dc_gan architecture
def define_gan(generator, discriminator):
    discriminator.trainable = False

    model = Sequential()

    model.add(generator)

    model.add(discriminator)

    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0002))
    return model

def generate_real_samples(dataset, n_samples):
    index = randint(0, dataset.shape[0], n_samples)
    X = dataset[index]
    y = np.ones((n_samples, 1))
    return X, y

def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

def save_plot(examples, epoch, n=7):
    examples = (examples + 1) / 2.0
    for i in range(n*n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis("off")
        pyplot.imshow(examples[i])
    filename = "CIFAR10_plot_LatentDim25_e%03d.png" % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

def save_model(epoch, generator, discriminator, dataset, latent_dim, n_samples=150):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n_samples)
    save_plot(x_fake, epoch)
    filename = "CIFAR10_model_LatentDim25_%03d.h5" % (epoch+1)
    generator.save(filename)

def train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        print(i)
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = discriminator.train_on_batch(X_real, y_real)
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss2, _ = discriminator.train_on_batch(X_fake, y_fake)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print(">%d, %d/%d" %
                (i+1, j+1, bat_per_epo))
        if (i+1) % 10 == 0:
            save_model(i, generator, discriminator, dataset, latent_dim)
            
latent_dim = 50

discriminator = discriminator_model()

generator = generator_model(latent_dim)

dc_gan = define_gan(generator, discriminator)

cifar10_data = ((x_train - 127.5)/127.5).astype("float32")

train(generator, discriminator, dc_gan, cifar10_data, latent_dim)

In [ ]:
from keras.models import load_model
from matplotlib import pyplot

def create_plot(examples, n):
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis("off")
        pyplot.imshow(examples[i, :, :])
    pyplot.show()

model = load_model("CIFAR10_model_010.h5")
latent_points = generate_latent_points(100, 100)
X = model.predict(latent_points)
X = (X + 1) / 2.0
#create_plot(X, 10)

pyplot.imshow(X[14])
filename = "CIFAR10_kernel3.png"
pyplot.savefig(filename)

In [ ]:
import math
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import mnist
import skimage as skim
import skimage.transform

def inception(dataset, epsilon):
    inception_model = InceptionV3()
    
    inception_scores = []
    for i in range(10):
        batch = dataset[i*math.floor(dataset.shape[0]/10):(i+1)*math.floor(dataset.shape[0]/10)]
        batch = batch.astype("float32")
        
        # resize the images for the inception_model
        copies = []
        for image in batch:
                copies.append(skim.transform.resize(image, (299,299,3)))
        batch = copies
        
        batch = preprocess_input(np.asarray(batch))
        
        prob_yx = inception_model.predict(batch)
        prob_y = expand_dims(prob_yx.mean(axis=0), 0)

        first_calc = prob_yx * (np.log(prob_yx + epsilon) - np.log(prob_y + epsilon))
        
        inception_score = np.exp(np.mean(first_calc.sum(axis=1)))
        
        inception_scores.append(inception_score)

    return np.mean(inception_scores)

(real_images, _), (_, _) = mnist.load_data()
score = inception(real_images[0:100], 1E-16)
generated_score = inception(X, 1E-16)
noisy = np.random.rand(32,32,3)
noisy_score = inception(noisy, 1E-16)
print("Generator stride of 2:")
print("Real MNIST score: ", score)
print("Generated MNIST score: ", generated_score)
print("Random noise score: ", noisy_score)